In [71]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn import metrics
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

In [98]:
test = pd.read_csv("test_transformed_allFeatures.csv")
df = pd.read_csv("train_transformed_allFeat.csv")
test.drop(test.columns[[7,13,14]], axis=1, inplace=True)
df.drop(df.columns[[7,13,14]], axis=1, inplace=True)

In [99]:
df.iloc[:,4:]

,aa,ra,jc,path_len,dc_A,dc_B,cc_A,cc_B
0,2.471649,0.531746,0.428571,1,0.002124,0.003186,0.187879,0.223134
1,2.976054,0.668651,0.466667,1,0.002124,0.003717,0.187879,0.209683
2,2.899858,0.628968,0.700000,1,0.002124,0.002390,0.187879,0.187891
3,2.471649,0.531746,0.600000,1,0.002124,0.002124,0.187879,0.181356
4,2.976054,0.668651,0.466667,1,0.002124,0.003717,0.187879,0.208769
...,...,...,...,...,...,...,...,...
32067,0.000000,0.000000,0.000000,3,0.002655,0.001328,0.246668,0.225652
32068,0.000000,0.000000,0.000000,999,0.002655,0.001593,0.246668,0.173938
32069,0.000000,0.000000,0.000000,3,0.002655,0.002390,0.246668,0.224344
32070,0.000000,0.000000,0.000000,3,0.002655,0.001062,0.246668,0.213252


In [100]:
test.iloc[:,4:]

,aa,ra,jc,path_len,dc_A,dc_B,cc_A,cc_B
0,0.000000,0.000000,0.000000,3,0.002124,0.001859,0.187879,0.218136
1,0.000000,0.000000,0.000000,5,0.002124,0.000797,0.187879,0.203260
2,0.000000,0.000000,0.000000,3,0.004249,0.008232,0.229441,0.256824
3,1.242670,0.400000,0.080000,1,0.006373,0.000797,0.263182,0.206104
4,0.000000,0.000000,0.000000,4,0.004514,0.006107,0.247700,0.246366
...,...,...,...,...,...,...,...,...
1995,0.000000,0.000000,0.000000,5,0.000266,0.000531,0.177465,0.217868
1996,0.000000,0.000000,0.000000,999,0.000531,0.000266,0.228538,0.000266
1997,0.000000,0.000000,0.000000,3,0.001062,0.007700,0.214815,0.249028
1998,1.766048,0.424368,0.053333,1,0.019649,0.001328,0.280597,0.231677


In [101]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,4:], ["pos" if i <= df.shape[0]//2 else "neg" for i in range(df.shape[0])])

In [102]:
LR = LogisticRegression(n_jobs = 4).fit(x_train, y_train)
LRT = LR.predict(x_test)

print(metrics.classification_report(y_test, LRT))


              precision    recall  f1-score   support

         neg       1.00      1.00      1.00      4017
         pos       1.00      1.00      1.00      4001

    accuracy                           1.00      8018
   macro avg       1.00      1.00      1.00      8018
weighted avg       1.00      1.00      1.00      8018



In [103]:
probs = LR.predict_proba(x_test)

In [104]:
metrics.roc_auc_score(y_test, probs[:, 1])

1.0

In [105]:
import collections
collections.Counter(LR.predict(test.iloc[:,4:]))

Counter({'neg': 1252, 'pos': 748})

In [106]:
collections.Counter(y_test)

Counter({'pos': 4001, 'neg': 4017})

In [107]:
collections.Counter(LR.predict(x_test))

Counter({'pos': 4001, 'neg': 4017})

In [108]:
results = LR.predict_proba(test.iloc[:,4:])
results[:,1]

array([7.66620525e-04, 1.06927707e-09, 7.71304714e-04, ...,
       7.70006621e-04, 9.99920385e-01, 1.06946092e-09])

In [109]:
output = pd.DataFrame({'Id': [i for i in range(1,2001)], 'Predicted': results[:,1]})
output.to_csv('logRegResult_no_btw_no_pa.csv', header = True, index= False)

In [ ]:
#ridge regression

clf = Ridge(alpha=1.0)
clf.fit(X, y)

Support Vector Machine (SVM)

In [57]:
SVM = LinearSVC(max_iter=10000)
#SVM_predict = SVM.predict(x_test)
clf = CalibratedClassifierCV(SVM) 
clf.fit(x_train, y_train)
clf_predict = clf.predict(x_test)


print(metrics.classification_report(y_test, clf_predict))

C:\Users\Melissa Speer\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Melissa Speer\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Melissa Speer\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\Melissa Speer\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


              precision    recall  f1-score   support

         neg       1.00      1.00      1.00      4041
         pos       1.00      1.00      1.00      3977

    accuracy                           1.00      8018
   macro avg       1.00      1.00      1.00      8018
weighted avg       1.00      1.00      1.00      8018



C:\Users\Melissa Speer\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [58]:
probs = clf.predict_proba(x_test)

In [59]:
probs

array([[0.67533204, 0.32466796],
       [0.34029695, 0.65970305],
       [0.35430718, 0.64569282],
       ...,
       [0.80227792, 0.19772208],
       [0.67984221, 0.32015779],
       [0.67347837, 0.32652163]])

In [60]:
metrics.roc_auc_score(y_test, probs[:, 1])

1.0

In [61]:
collections.Counter(clf.predict(test.iloc[:,4:]))

Counter({'neg': 1465, 'pos': 535})

In [42]:
collections.Counter(y_test)

Counter({'neg': 4029, 'pos': 3989})

In [20]:
results = clf.predict_proba(test.iloc[:,4:])
results[:,1]

array([0.32588784, 0.11201567, 0.32562297, ..., 0.32601893, 0.65587601,
       0.11203549])

In [22]:
output = pd.DataFrame({'Id': [i for i in range(1,2001)], 'Predicted': results[:,1]})
output.to_csv('svmResult.csv', header = True, index= False)